[View in Colaboratory](https://colab.research.google.com/github/maddy46/mlblr4a_experiments/blob/master/DNST_CIFAR10_AUG_JIC.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
!rm utils.py
!wget https://raw.githubusercontent.com/Zahlii/colab-tf-utils/master/utils.py
import utils
import os
import keras

def compare(best, new):
  return best.losses['val_acc'] < new.losses['val_acc']

def path(new):
  if new.losses['val_acc'] > 0.2:
    return '4B_008_%s.h5' % new.losses['val_acc']


  
saver = utils.GDriveCheckpointer(compare,path)

In [5]:
# Hyperparameters
num_classes = 10
epochs =20
num_filter = 64
compression = 0.5
dropout_rate = 0.25
l = 6
batch_size = 32

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print (img_height)
print (img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 11s 0us/step
32
32


In [7]:
print (x_train.shape[0])
print (x_test.shape[0])

50000
10000


In [8]:

new_images = np.empty((50000, 24, 24, 3), dtype='uint8')
new_test = np.empty((10000, 24, 24, 3), dtype='uint8')

for image_count in range(x_train.shape[0]):
    new_images[image_count] = cv2.resize(x_train[image_count], (24, 24))
for image_count1 in range(x_test.shape[0]):
    new_test[image_count1] = cv2.resize(x_test[image_count1], (24, 24))
img_height1, img_width1, channel1 = new_images.shape[1],new_images.shape[2],new_images.shape[3]
print (img_height1)
print (img_width1)

print('x resizing Done.')


24
24
x resizing Done.


In [9]:
datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=20,width_shift_range=0.2,height_shift_range=0.2)
datagen.fit(new_images)

In [10]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [11]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [12]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [13]:

input = Input(shape=(img_height1, img_width1, channel1,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)


3


In [14]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 24, 24, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 24, 24, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [15]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
'''
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=cb)
                    '''

model.fit_generator(datagen.flow(new_images, y_train, batch_size=batch_size), epochs=epochs, 
                    verbose=1, validation_data=(new_test
                                                , y_test),
                    steps_per_epoch=len(new_images)/batch_size, callbacks=[saver])

Epoch 1/20
 275/1562 [====>.........................] - ETA: 1:16:45 - loss: 1.9553 - acc: 0.2715

In [19]:
# Save the trained weights in to .h5 format
model.save_weights("saved_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=20,width_shift_range=0.2,height_shift_range=0.2)
datagen.fit(x_train)

In [18]:
input_original = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input_original)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)

3


In [19]:
model = Model(inputs=[input_original], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_3 (

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [21]:
model.load_weights("saved_model.h5",by_name=True)

OSError: ignored

In [0]:
epochs = 30
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, 
                    verbose=1, validation_data=(x_test
                                                , y_test),
                    steps_per_epoch=len(x_train)/batch_size, callbacks=[saver])

In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])